In [66]:
from gensim.models import fasttext as ft
import numpy as np
import os
import nltk
from nltk.corpus import stopwords
import operator
import re

In [2]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yoshi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yoshi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# https://fasttext.cc/docs/en/crawl-vectors.html
model=ft.load_facebook_model(os.path.join(os.getcwd(),"binaries/data/cc.en.300.bin"))

C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [4]:
window_size=3
stemmer = nltk.wordnet.WordNetLemmatizer()

In [75]:
def rank(text):
    text=text.lower()
    stops = set(stopwords.words("english"))
    tokens = nltk.word_tokenize(text)
    filtered_words = [stemmer.lemmatize(word) for word in tokens if word not in stops]
    scores=dict()
    counts=dict()
    contextSizes=dict()
    for i in range(len(filtered_words)):
        counts.setdefault(filtered_words[i],0)
        counts[filtered_words[i]]+=1
        leftBound=max(0,i-window_size)
        rightBound=min(len(filtered_words)-1,i+window_size)
        contextSize=rightBound-leftBound+1
        contextSizes.setdefault(filtered_words[i],contextSize)
        contextSizes[filtered_words[i]]=max(contextSizes[filtered_words[i]],contextSize)
        for j in range(i+1,rightBound+1):
            similarity_score=model.wv.similarity(filtered_words[i],filtered_words[j])
            similarity_score=(similarity_score+1)/2.0
            scores.setdefault(filtered_words[i],similarity_score)
            scores.setdefault(filtered_words[j],similarity_score)
            scores[filtered_words[i]]=min(scores[filtered_words[i]],similarity_score)
            scores[filtered_words[j]]=min(scores[filtered_words[j]],similarity_score)
    wordScores=list(map(list, scores.items()))
    for i in range(len(wordScores)):
        wordScores[i][1]=wordScores[i][1]/(counts[wordScores[i][0]]*contextSizes[wordScores[i][0]])
    wordScores=sorted(wordScores, key = lambda x: x[1])#,reverse=True)
    filteredWordScores=[]
    for word in wordScores:
        filtered=re.sub('[^a-zA-Z]', '', word[0])
        if(len(filtered)>2):
            filteredWordScores.append([filtered,word[1]])
    return filteredWordScores

In [76]:
rank("Machine learning (ML) is the scientific study of algorithms and statistical models that computer systems use to perform a specific task without using explicit instructions, relying on patterns and inference instead. It is seen as a subset of artificial intelligence. Machine learning algorithms build a mathematical model based on sample data, known as training data, in order to make predictions or decisions without being explicitly programmed to perform the task. Machine learning algorithms are used in a wide variety of applications, such as email filtering and computer vision, where it is difficult or infeasible to develop a conventional algorithm for effectively performing the task.")

[['microsoft', 0.01766083509262119],
 ['software', 0.024401567920687654],
 ['product', 0.03671004609870059],
 ['computer', 0.037528068625501225],
 ['office', 0.037954039339508326],
 ['world', 0.03898749394076211],
 ['corporation', 0.040575627769742696],
 ['largest', 0.07049686554819345],
 ['known', 0.07064334037048477],
 ['revenue', 0.07106005606640663],
 ['explorer', 0.07256370822765998],
 ['browser', 0.07279257848858833],
 ['operating', 0.07289374406848635],
 ['license', 0.07314764295837708],
 ['washington', 0.07317898542221103],
 ['develops', 0.07317898542221103],
 ['tablet', 0.07325364662600416],
 ['hardware', 0.07331339030393533],
 ['support', 0.07364704433296408],
 ['measured', 0.07371565060956138],
 ['series', 0.07409222290984221],
 ['suite', 0.07427640418921198],
 ['flagship', 0.07434462888964585],
 ['headquartered', 0.07440065246607576],
 ['one', 0.07440400230033058],
 ['maker', 0.07464030704327992],
 ['surface', 0.07479857041367463],
 ['best', 0.07493438332208566],
 ['xbox', 